In [1]:
# from catboost import CatBoostClassifier, Pool
# import pandas as pd
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import accuracy_score
# from sklearn.metrics import confusion_matrix

# pd.set_option('display.max_columns', None)
# data_raw = pd.read_csv('prepped_data.csv', low_memory=False)
# # data_raw = data_raw.dropna(axis=1)

# columns_to_drop = ['Unnamed: 0', 'policy_nr_hashed', 'last_data_year', 'first_data_year', 'control_group', 'last_brand', 'last_type', 'last_fuel_type', 'last_product']
# data = data_raw.drop(columns=columns_to_drop, axis=1)

# y = data['churn']
# X = data.drop('churn', axis=1)


# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# model = CatBoostClassifier(iterations=100, 
#                            learning_rate=0.1, 
#                            depth=10, 
#                            loss_function='MultiClass')


# model.fit(X_train, y_train, verbose=False)

# predictions = model.predict(X_test)
# cm = confusion_matrix(y_test, predictions)

# print(cm)

In [2]:
# from catboost import CatBoostClassifier
# import pandas as pd
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import confusion_matrix

# pd.set_option('display.max_columns', None)
# data_raw = pd.read_csv('prepped_data.csv', low_memory=False)
# # data_raw = data_raw.dropna(axis=1)

# columns_to_drop = ['Unnamed: 0', 'policy_nr_hashed', 'last_data_year', 'first_data_year', 'control_group', 'last_brand', 'last_type', 'last_fuel_type', 'last_product']
# data = data_raw.drop(columns=columns_to_drop, axis=1)

# y = data['churn']
# X = data.drop('churn', axis=1)

# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# model = CatBoostClassifier(iterations=100,
#                            learning_rate=0.1,
#                            depth=10,
#                            loss_function='MultiClass',
#                            auto_class_weights='Balanced',  # This will handle the class imbalance
#                            random_state=42)  # For reproducibility

# model.fit(X_train, y_train, verbose=False)

# predictions = model.predict(X_test)
# cm = confusion_matrix(y_test, predictions)

# print(cm)


In [1]:
from catboost import CatBoostClassifier
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

pd.set_option('display.max_columns', None)
data_raw = pd.read_csv('prepped_data.csv', low_memory=False)
data_raw = data_raw.dropna(axis=1)

categorical_columns = ['last_brand', 'last_type', 'last_fuel_type']
data_raw = pd.get_dummies(data_raw, columns=categorical_columns)
columns_to_drop = ['Unnamed: 0', 'policy_nr_hashed', 'last_data_year', 'first_data_year', 'control_group', 'last_product']
data = data_raw.drop(columns=columns_to_drop, axis=1)

y = data['churn']
X = data.drop('churn', axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = CatBoostClassifier(iterations=100,
                           learning_rate=0.1,
                           depth=10,
                           loss_function='MultiClass',
                           auto_class_weights='Balanced',  # This will handle the class imbalance
                           random_state=42)  # For reproducibility

model.fit(X_train, y_train, verbose=False)

predictions = model.predict(X_test)
cm = confusion_matrix(y_test, predictions)

print(cm)

[[3759 1502]
 [ 168  474]]
